<a href="https://colab.research.google.com/github/RSid8/SMM4H21/blob/main/Task1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U spacy

     |████████████████████████████████| 12.8MB 235kB/s 
     |████████████████████████████████| 460kB 50.0MB/s 
     |████████████████████████████████| 1.1MB 49.1MB/s 
     |████████████████████████████████| 9.1MB 52.7MB/s 
     |████████████████████████████████| 122kB 55.5MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=8b3deabb1128bf67bd10e55d0adfb80d635044bdbeea27d9d3d4841f7bb65d7a
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: smart-open 4.2.0
    Uninstalling smart-open-4.2.0:
      Successfully uninstalled smart-open-4.2.0
  Found existing installation: thinc 7.4.0
    U

In [ ]:
!python -m spacy download 'en_core_web_trf'

2021-04-11 11:29:10.591276: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 459.7MB 32kB/s 
     |████████████████████████████████| 1.0MB 16.7MB/s 
     |████████████████████████████████| 1.8MB 52.4MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 870kB 50.4MB/s 
     |████████████████████████████████| 2.9MB 48.4MB/s 
  Created wheel for torchcontrib: filename=torchcontrib-0.0.2-cp37-none-any.whl size=7532 sha256=eec69707c6e0e37c3a706127d8210270f508858e48b6aaab550df5176a110b95
  Stored in directory: /root/.cache/pip/wheels/06/06/7b/a5f5920bbf4f12a2c927e438fac17d4cd9560f8336b00e9a99
  Created wheel for ftfy: filename=ftfy-5.9-cp37-none-any.whl size=46451 sha256=9d07173d72f56ef8c1492450bef5fae1a21b2e965d95b9fc658e3ba456ad5c37
  Stored in directory: /root/.cache/pip/wheels/5e/2e/f0/b07196e8c929114998f0316894a61c752b63bfa3fdd50d2fc

# Data Processing and Pre processing
Clean up the dataset by removing overlapping entities

In [ ]:
import pandas as pd
df_tweets = pd.read_csv("/content/drive/MyDrive/UPENN/Task1b/train_tweets.tsv", sep = '\t')
df_spans = pd.read_csv("/content/drive/MyDrive/UPENN/Task1b/train_spans.tsv", sep = '\t')
df_tweets.columns = ["tweet_id", "tweet"]
df_spans.columns = ["tweet_id", "label", "start", "end", "span"]
df_tweets.head()

,tweet_id,tweet
0,347806215776116737,"@ashleylvivian if #avelox has hurt your liver,..."
1,350336129817509888,"apparently, baclofen greatly exacerbates the ""..."
2,332540699692130304,pt of mine died from cipro rt @ciproispoison: ...
3,332542581760204800,@gastromom The only pt of mine who ever died w...
4,343944308828811264,owww. i hurt my foot. &amp; am concerned. 1st ...


In [ ]:
df_spans.head()

,tweet_id,label,start,end,span
0,347806215776116737,ADE,31,46,HURT YOUR Liver
1,350336129817509888,ADE,48,50,AD
2,350336129817509888,ADE,88,93,focus
3,332540699692130304,ADE,11,15,died
4,332542581760204800,ADE,40,44,died


In [ ]:
df = df_tweets.merge(df_spans, on = "tweet_id")
df.head()

,tweet_id,tweet,label,start,end,span
0,347806215776116737,"@ashleylvivian if #avelox has hurt your liver,...",ADE,31,46,HURT YOUR Liver
1,350336129817509888,"apparently, baclofen greatly exacerbates the ""...",ADE,48,50,AD
2,350336129817509888,"apparently, baclofen greatly exacerbates the ""...",ADE,88,93,focus
3,332540699692130304,pt of mine died from cipro rt @ciproispoison: ...,ADE,11,15,died
4,332542581760204800,@gastromom The only pt of mine who ever died w...,ADE,40,44,died


In [ ]:
len(df)

1691

In [ ]:
TRAIN_DATA = []
entity_list = []
for ind in df.index:
  id = df['tweet_id'][ind]
  x = df['tweet'][ind]
  if ind+1<len(df):
    next_id = df['tweet_id'][ind+1]
  else:
    next_id = None 
  if next_id == id:
    span_tuple = (df['start'][ind], df['end'][ind], df['label'][ind])
    entity_list.append(span_tuple)
  else:
    span_tuple = (df['start'][ind], df['end'][ind], df['label'][ind])
    entity_list.append(span_tuple)    
    train_sample = (x, {'entities': entity_list})
    TRAIN_DATA.append(train_sample)
    entity_list = []

In [ ]:
print(TRAIN_DATA)
print(len(TRAIN_DATA))

[('@ashleylvivian if #avelox has hurt your liver, avoid tylenol always, as it further damages liver, eat grapefruit unless taking cardiac drugs', {'entities': [(31, 46, 'ADE')]}), ('apparently, baclofen greatly exacerbates the "ad" part of my adhd. average length of focus today: about 30 seconds.', {'entities': [(48, 50, 'ADE'), (88, 93, 'ADE')]}), ('pt of mine died from cipro rt @ciproispoison: @gastromom if only more doctors thought like you! i lost my entire life to 12 cipro pills', {'entities': [(11, 15, 'ADE')]}), ('@gastromom The only pt of mine who ever died was one age 21 profound autism chronic underwt & GI issues, given lots of Cipro. Terrible', {'entities': [(40, 44, 'ADE')]}), ("owww. i hurt my foot. &amp; am concerned. 1st warning on cipro is tendon damage. :-/ it certainly wasn't a rupture. i'm sure i'm just paranoid.", {'entities': [(66, 79, 'ADE')]}), ("@bilgeebiri oh robitussin dreams are notorious. although what will really turn 'em into a bunuel movie is cipro.", {'e

# Creating the spaCy model


In [ ]:
import spacy
import warnings
from pathlib import Path
from spacy.util import minibatch, compounding
from spacy.training import Example
import random
from tqdm import tqdm
import torch

In [ ]:
model = None
output_dir = Path("/content/model_val")
n_iter=250

In [ ]:
nlp2 = spacy.load('en_core_web_trf')
#nlp2 = spacy.load('en_core_web_sm')
is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    torch.set_default_tensor_type("torch.cuda.FloatTensor")


In [ ]:
if model is not None:
    ner_model = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
# this will create a blank english model
    ner_model = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

Created blank 'en' model


In [ ]:
# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in ner_model.pipe_names:
  ner = ner_model.create_pipe("ner")
  ner_model.add_pipe("ner", last=True)
# otherwise, get it so we can add labels
else:
  ner = ner_model.get_pipe("ner")


In [ ]:
for _,annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

In [ ]:
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in ner_model.pipe_names if pipe not in pipe_exceptions]
# only train NER
with ner_model.disable_pipes(*other_pipes), warnings.catch_warnings():
#with nlp2.disable_pipes(*other_pipes), warnings.catch_warnings():
  # ignore warnings for misaligned entity spans once
  warnings.filterwarnings("ignore", category=UserWarning, module='spacy')

# reset and initialize the weights randomly – but only if we're
# training a new model
  if model is None:
    optimizer = ner_model.begin_training()
    #optimizer = nlp2.resume_training()
    for i in tqdm(range(n_iter), total = n_iter):
      random.shuffle(TRAIN_DATA)
      losses = {}
      batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
      for batch in batches:
        examples = []
        for text, annots in batch:
          examples.append(Example.from_dict(ner_model.make_doc(text), annots))
        ner_model.update(examples, drop = 0.3, sgd = optimizer, losses = losses)
        #nlp2.update(examples, drop = 0.5, sgd = optimizer, losses = losses)
      print("Epoch, Losses:",i, losses)

  0%|          | 1/250 [00:14<1:00:42, 14.63s/it]

Epoch, Losses: 0 {'ner': 2769.8204395220732}


  1%|          | 2/250 [00:27<58:17, 14.10s/it]  

Epoch, Losses: 1 {'ner': 1878.3243639311374}


  1%|          | 3/250 [00:39<55:29, 13.48s/it]

...

Epoch, Losses: 247 {'ner': 57.009245359012205}


100%|█████████▉| 249/250 [51:17<00:10, 10.96s/it]

Epoch, Losses: 248 {'ner': 83.49762709669679}


100%|██████████| 250/250 [51:28<00:00, 12.36s/it]

Epoch, Losses: 249 {'ner': 82.37191147339722}


In [ ]:
if output_dir is not None:
    output_dir = Path(output_dir)
if not output_dir.exists():
    output_dir.mkdir()
ner_model.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to /content/model_val


In [ ]:
# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Loading from /content/model_val
Entities [('drowsy', 'ADE'), ('haze', 'ADE')]
Entities [('cravings', 'ADE')]
Entities [('drowsy', 'ADE')]
Entities [('numb', 'ADE')]
Entities [('barely function', 'ADE')]
Entities [('lucidity', 'ADE')]
Entities [('jolting', 'ADE'), ('rocking', 'ADE')]
Entities [('withdrawal', 'ADE')]
Entities [('sleep for 12 straight hours', 'ADE')]
Entities [('asleep', 'ADE')]
Entities [('sleep on my own. but i got at least 16 hours', 'ADE')]
Entities [('tearful', 'ADE'), ('poor nights sleep', 'ADE'), ('withdrawing', 'ADE')]
Entities [('rage', 'ADE')]
Entities [('aches', 'ADE')]
Entities [('pains', 'ADE'), ('fluid retention', 'ADE')]
Entities [('high', 'ADE')]
Entities [('wired', 'ADE'), ('not sleeping', 'ADE')]
Entities [('high', 'ADE'), ('feel like throwing up', 'ADE')]
Entities [('sleep for 11 hours', 'ADE')]
Entities [('Coming off venlafaxine', 'ADE'), ('nasty', 'ADE')]
Entities [('addiction', 'ADE')]
Entities [('neuro problems', 'ADE'), ('ataxia', 'ADE')]
Entities 

In [ ]:
df_val = pd.read_csv("/content/tweets.tsv", sep = '\t')
df_val.head()
print(len(df_val))

914


In [ ]:
!zip -r /content/spacymodel_sm.zip /content/model_sm/

	zip warning: name not matched: /content/model_sm/

zip error: Nothing to do! (try: zip -r /content/spacymodel_sm.zip . -i /content/model_sm/)


In [ ]:
id = []
start = []
end = []
word = []
for i in df_val.index:
  doc = nlp2(df_val["tweet"][i])
  for ent in doc.ents:
    id.append(df_val["tweet_id"][i])
    start.append(ent.start_char)
    end.append(ent.end_char)
    word.append(ent.text)
df_1b = pd.DataFrame(list(zip(id, start, end, word)), columns = ['tweet_id', 'start', 'end', 'span'])
df_1b.head()

,tweet_id,start,end,span
0,342138828087234560,125,138,muscle spasms
1,342382016030973953,61,76,gaining. stupid
2,342382016030973953,91,110,gain like 50 pounds
3,345414697148887041,87,103,continuous sleep
4,345414697148887041,126,134,headache


In [ ]:
len(df_1b)

374

In [ ]:
df_1b.to_csv("1b_val_3.tsv", sep = '\t')

#Added this section to get validation scores


In [ ]:
df_1a = pd.read_csv('/content/val_final.tsv', sep = '\t') 
df_1b = pd.read_csv('/content/1b_val_3.tsv', sep = '\t')
df_1b.drop(["Unnamed: 0"], axis=1, inplace=True)
df_1b.head()

,tweet_id,start,end,span
0,342138828087234560,125,138,muscle spasms
1,342382016030973953,61,76,gaining. stupid
2,342382016030973953,91,110,gain like 50 pounds
3,345414697148887041,87,103,continuous sleep
4,345414697148887041,126,134,headache


In [ ]:
df_1 = df_1a.merge(df_1b, on = 'tweet_id')
df_1.columns = ["tweet_id", "label", "start", "end", "span"]
df_1["start"] = df_1["start"].astype(int)
df_1["end"] = df_1["end"].astype(int)
df_1.dropna(axis=0, inplace=True)
df_1 = df_1[df_1.label=="ADE"]
df_1.head()

,tweet_id,label,start,end,span
0,342138828087234560,ADE,125,138,muscle spasms
1,342382016030973953,ADE,61,76,gaining. stupid
2,342382016030973953,ADE,91,110,gain like 50 pounds
3,345414697148887041,ADE,87,103,continuous sleep
4,345414697148887041,ADE,126,134,headache


In [ ]:
df_1.to_csv('/content/val_score_3.tsv', sep = '\t', index= False)